Here we'll show a bit about how the convex hull algorithm works in 3D now.

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=1729)
num_points = 40
X = rng.normal(size=(num_points, 3))

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
fig, axes = plt.subplots(subplot_kw={"projection": "3d"})
axes.scatter(*X.T);

In [ ]:
import zmsh
hull_machine = zmsh.ConvexHullMachine(X)
geometry = hull_machine.geometry

In [ ]:
from copy import deepcopy
geometries = [deepcopy(hull_machine.geometry)]
visibility_graphs = [hull_machine.visible]

while not hull_machine.is_done():
    hull_machine.step()
    geometries.append(deepcopy(hull_machine.geometry))
    visibility_graphs.append(deepcopy(hull_machine.visible))

Now we'll visualize the state of the algorithm at every step.
The blue point is known to be an extreme point, so we'll add it to the hull next.
The orange faces are visible to this new extreme point; they'll be split at the next iteration.

In [ ]:
from ipywidgets import interact
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

@interact(step=(0, len(geometries) - 1))
def f(step=0):
    ax.clear()
    geometry = geometries[step]
    
    visible = visibility_graphs[step]
    num_cells = visible.shape[1]
    num_visible_vertices = np.array(
        [visible.getcol(cell_id).nnz for cell_id in range(num_cells)]
    )
    cell_id = np.argmax(num_visible_vertices)
    vertex_id = np.argmin(visible.getcol(cell_id))
    cell_ids = visible.getrow(vertex_id).nonzero()[1]

    colors = np.full(num_cells, "tab:blue", dtype=object)
    colors[cell_ids] = "tab:orange"
    zmsh.visualize(geometry, dimension=2, colors=colors, ax=ax)
    zmsh.visualize(geometry, dimension=1, ax=ax)
    
    colors = np.full(X.shape[0], "black", dtype=object)
    colors[vertex_id] = "tab:blue"
    zmsh.visualize(geometry, dimension=0, colors=colors, ax=ax)

Computing convex hulls in dimension 3 or higher is much more difficult in the 2D case because now multiple hull faces can be visible to the same extreme point.